## Задание 1.

Дана строка s. Найдите длину самой длинной подстроки без повторяющихся символов. 
Пример:
s = “aabchbad”. Ответ: 5 
Альтернативное объяснение:
Подстрока - непрерывный срез строки, то есть для строки: “helloeveryone”:

1. “hello” - подстрока
2. “every” - подстрока
3. “helloeveryone” - подстрока
4. “helone” (“helloeveryone”) - не подстрока

Вам необходимо найти максимальную длину подстроки, внутри которой не будет повторяющихся символов.

## Решение:

In [1]:
import time
from functools import wraps
import pytest

In [2]:
def timing(func):
    '''Вычисляет время работы функции''' 
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()  
        result = func(*args, **kwargs)  
        end_time = time.time()
        elapsed_time = end_time - start_time  
        print(f"Function {func.__name__} took {elapsed_time:.9f} seconds")
        return result
    return wrapper

In [3]:
@timing
def length_of_longest_substring(s: str) -> int:
    if not isinstance(s, str):
        raise TypeError('Input must be a string.')

    symbol_index_map: dict = {}
    max_length: int = 0
    start: int = 0
    
    for end, char in enumerate(s):
        if char in symbol_index_map and symbol_index_map[char] >= start:
            start = symbol_index_map[char] + 1
        symbol_index_map[char] = end
        max_length = max(max_length, end - start + 1)
    
    return max_length


In [4]:
s = "aboBa"*100500
print(length_of_longest_substring(s))

Function length_of_longest_substring took 0.129019260 seconds
4


### Комментарий

В задании не указано, как обрабатывать регистры, поэтому в моей реализации символы `А` и `а`, а также любая другая пара строчных и прописных букв считаются разными символами.

## Объяснение:

1. Инициализация: 
    - `symbol_index_map`  <-- {}. Хэш таблица вида {symbol:index_number} - хранит последний индекс каждого символа
    - `max_length` <-- 0. Длина макс. подстроки
    - `start` <-- 0. индекс начала подстроки (текущего окна)
2. Проходимся по каждому символу char, используя индекс end. Это текущая позиция символа char 
3. Если символ уже встречался и его индекс находится в текущем окне, значит, что есть повтор, и начало окна нужно сдвинуть на следующую позицию (на следующую после той, которая пока записана в хэш-таблице)
4. Обновляем последний индекс текущего символа в хэш таблице
5. Считаем максимальную длину подстроки. Обновляем только в том случае, если она больше предыдущей максимальной.

Изначально я хотел ~~выпендриться и~~ вместо условия 
- `if char in symbol_index_map and symbol_index_map[char] >= start`:

Написать условие 
- `if symbol_index_map.get(char, -1) >= start`:

Но первый вариант в теории быстрее для интерпретатора python, т.к. get() обрабатывает дополнительную логику вовращения значения по умолчанию.
По итогу проверил на строке `"abcabcbb" * 100` на миллионе итераций, результаты:

- Average time for `get` method: 0.0001914035 seconds
- Average time for `in` method: 0.0001909332 seconds

## Сложность

- Алгоритмическая сложность по памяти `O(n)`, n - длина строки. Так как проходимся по каждому элементу 1 раз.
- По памяти в худшем случае `O(n)` - если размер словаря будет равен длине строки (то есть все символы в строке уникальные). В лучшем случае `O(m)` - где m - количество уникальных символов в строке. Основным фактором явяляется словарь, константами О(1) можно принебречь.

## Тесты

1. `Пустая строка.` Базовый краевой случай. При разработке каких-то сложных алгоритмов разработчики могут забыть про то, что среди входных данных может быть пустая строка.
2. `Очень длинная строка.` Тоже базовый краевой случай. Если бы писали на Си, то размер входной строки был бы ограничен размером типа данных переменных, в которых хранятся индексы. Иначе возможно переполнение регистра при записи слишком большого числа, из-за чего отсчёт бы начался заново и функция бы работала некорректно. Проще говоря, если бы индекс был типа unsigned char, то при попытке проиндексировать 256-й элемент строки, проиндексировался бы 0-й элемент. В питоне такой проблемы нет, то всё же проверить стоит.
3. `Строка с однимволом.` На таких тестах очень часто падают функции, потому что разработчики при написании алгоритма отталкиваются от каких-то более распространённых юзер-кейсов и могут забить на нулевую или минимальную строку.
4. `Строка с одинаковыми символами.` Такеже граничный кейс, важно, чтобы алгоритм корректно обрабатывал строки с отсутствием уникальных символов.
5. `Строка с уникальными символами + различные символы.` Помогает убедиться, что функция работает корректно с разнообразными символами и с уникальными символами.
6. `Неправильный тип входных данных`. Всё понятно из названия

In [5]:
extended_alphabet = (
        ''.join(chr(i) for i in range(65, 91)) +  # A-Z
        ''.join(chr(i) for i in range(97, 123)) +  # a-z
        ''.join(chr(i) for i in range(1040, 1072)) +  # А-Я 
        ''.join(chr(i) for i in range(1072, 1104)) +  # а-я
        '0123456789' +  # цифры
        "!@#$%^& *()\/"  # спец символы
    )

In [6]:
@timing
def test_length_of_longest_substring():
    assert length_of_longest_substring("") == 0, "Failed on empty string"
    assert length_of_longest_substring("abc" * (10**7)  ) == 3, "Failed on very long string test"
    assert length_of_longest_substring("a") == 1, "Failed on single character"
    assert length_of_longest_substring("aaaaaa") == 1, "Failed on string with all same characters"
    assert length_of_longest_substring(extended_alphabet) == len(extended_alphabet), "Failed on extended alphabet test"  

    # тесты на неправильные типы данных
    with pytest.raises(TypeError) as excinfo:
        length_of_longest_substring(123)
    assert str(excinfo.value) == "Input must be a string.", "Failed on type check for integer input"
    
    with pytest.raises(TypeError) as excinfo:
        length_of_longest_substring([])
    assert str(excinfo.value) == "Input must be a string.", "Failed on type check for list input"
    
    with pytest.raises(TypeError) as excinfo:
        length_of_longest_substring(None)
    assert str(excinfo.value) == "Input must be a string.", "Failed on type check for None input"  
    print("All tests passed.")

In [7]:
test_length_of_longest_substring()

Function length_of_longest_substring took 0.000000000 seconds
Function length_of_longest_substring took 8.354145050 seconds
Function length_of_longest_substring took 0.000000000 seconds
Function length_of_longest_substring took 0.000000000 seconds
Function length_of_longest_substring took 0.000000000 seconds
All tests passed.
Function test_length_of_longest_substring took 8.368297577 seconds
